In [7]:
from ipfx_new.ipfx_new.x_to_nwb import ABFConverter_fix
ABFConverter_fix.ABFConverter('examples/CC_2.abf', 'output/CC_2.nwb',includeChannelList=["IN 0"], acquisitionChannelName=["IN 1"], stimulusChannelName= ['IN 0'])

C:\Users\Allose\Documents\GitHub\ABF2_NWB\ipfx_new\ipfx_new\x_to_nwb\ABFConverter_fix.py:161: UserWarning: Could not find the JSON file examples/CC_2.json with settings.
  warnings.warn(f"Could not find the JSON file {settings} with settings.")
C:\Users\Allose\Documents\GitHub\ABF2_NWB\ipfx_new\ipfx_new\x_to_nwb\ABFConverter_fix.py:473: UserWarning: Could not find the scale factor for the stimset cc step 2, using 1.0 as fallback.
  warnings.warn(f"Could not find the scale factor for the stimset {stimset}, using {DEFAULT_SCALE_FACTOR} as fallback.")
C:\Users\Allose\Documents\GitHub\ABF2_NWB\ipfx_new\ipfx_new\x_to_nwb\ABFConverter_fix.py:500: UserWarning: Could not find settings for amplifier unknown of channel IN 0.
  warnings.warn(f"Could not find settings for amplifier {amplifier} of channel {adcName}.")


In [8]:
# Load IPFX libraries
from ipfx.dataset.create import create_ephys_data_set
from ipfx.data_set_features import extract_data_set_features
from ipfx.utilities import drop_failed_sweeps
from ipfx.qc_feature_extractor import sweep_qc_features

# Load pandas library too
import pandas as pd
from ipfx.dataset.create import create_ephys_data_set
from ipfx.utilities import drop_failed_sweeps

file_name = "output/CC_2.nwb"

# Create Ephys Data Set

print('loading dataset into data structure...')
#nwb_file_name = '/Users/stripathy/Downloads/sub-601462951_ses-601810292_icephys.nwb'
data_set = create_ephys_data_set(nwb_file=file_name, ontology="ipfx/ipfx/defaults/stimulus_ontology.json") ##loads nwb file into ipfx data structure

# sweep = data_set.sweep(0)
# sweep_features = sweep_qc_features(data_set)
# Drop failed sweeps: sweeps with incomplete recording or failing QC criteria
drop_failed_sweeps(data_set)
# ipfx.data_set_features.extract_data_set_features(data_set, subthresh_min_amp=None)[source]


loading dataset into data structure...


C:\Users\Allose\Anaconda3\lib\site-packages\ipfx\stimulus.py:88: UserWarning: Could not find stimulus: cc step 2
  warnings.warn("Could not find stimulus: %s" % tag)


In [10]:
from ipfx.feature_extractor import SpikeFeatureExtractor, SpikeTrainFeatureExtractor

import ipfx.stimulus_protocol_analysis as spa
from ipfx.epochs import get_stim_epoch
import matplotlib.pyplot as plt


# get sweep table of Long Square sweeps
long_square_table = data_set.filtered_sweep_table(
    stimuli=data_set.ontology.long_square_names
)
long_square_sweeps = data_set.sweep_set(long_square_table.sweep_number)

long_square_sweeps = data_set.sweep_set()

# Select epoch corresponding to the actual recording from the sweeps
# and align sweeps so that the experiment would start at the same time
long_square_sweeps.select_epoch("recording")
#long_square_sweeps.align_to_start_of_epoch("experiment")

# find the start and end time of the stimulus
# (treating the first sweep as representative)
stim_start_index, stim_end_index = get_stim_epoch(long_square_sweeps.i[0], test_pulse=False)
stim_start_time = long_square_sweeps.t[0][stim_start_index]
stim_end_time = long_square_sweeps.t[0][stim_end_index]

# stim_start_time = 0.1
# stim_end_time = 0.8

# build the extractors
spfx = SpikeFeatureExtractor(start=stim_start_time, end=stim_end_time, filter = 1)
sptfx = SpikeTrainFeatureExtractor(start=stim_start_time, end=stim_end_time)

# run the analysis and print out a few of the features
long_square_analysis = spa.LongSquareAnalysis(spfx, sptfx, subthresh_min_amp=-100.0)
data = long_square_analysis.analyze(long_square_sweeps)

fields_to_print = [
    'v_baseline',
    'vm_for_sag',
    'sag'
]

for field in fields_to_print:
    print("%s: %s" % (field, str(data[field])))

INFO:root:Need to recalculate threshold-peak pair that exceeds maximum allowed interval (0.005000 s)
INFO:root:Could not redetermine threshold-peak pair - dropping that pair
C:\Users\Allose\Anaconda3\lib\site-packages\ipfx\spike_features.py:87: RuntimeWarning: invalid value encountered in less
  width_levels[width_levels < v[spike_indexes]] = thresh_to_peak_levels[width_levels < v[spike_indexes]]
INFO:root:Need to recalculate threshold-peak pair that exceeds maximum allowed interval (0.005000 s)
INFO:root:Could not redetermine threshold-peak pair - dropping that pair
INFO:root:Need to recalculate threshold-peak pair that exceeds maximum allowed interval (0.005000 s)
INFO:root:Could not redetermine threshold-peak pair - dropping that pair
INFO:root:Need to recalculate threshold-peak pair that exceeds maximum allowed interval (0.005000 s)
INFO:root:Could not redetermine threshold-peak pair - dropping that pair
INFO:root:Need to recalculate threshold-peak pair that exceeds maximum allowed

v_baseline: -67.77605260213217
vm_for_sag: -90.271
sag: 0.08304978907108307
